In [24]:
from pathlib import Path
import torch
import tqdm

from au2v.config import ModelConfig, TrainerConfig
from au2v.dataset_manager import load_dataset_manager
from au2v.trainer import PyTorchTrainer
from au2v.model import load_model, PyTorchModel

In [25]:
def calc_accuracy(
    model: PyTorchModel,
    num_item: int,
    test_dataset: list[tuple[int, list[int], list[int]]],
    top_k: list[int],
) -> dict[str, float]:
    hit_counts = {k: 0 for k in top_k}
    for seq_index, context_items, target_indices in tqdm.tqdm(test_dataset):
        rec_list = model.output_rec_lists(
            seq_index=torch.LongTensor([seq_index]),
            item_indices=torch.LongTensor([context_items]),
            cand_item_indices=torch.arange(num_item),
            k=max(top_k),
        )
        for k in top_k:
            hit_counts[k] += len(set(target_indices) & set(rec_list[0][:k]))

    total_rec = len(test_dataset)
    results = {
        f"Accuracy@{k}": hit_count / total_rec / k
        for k, hit_count in hit_counts.items()
    }
    return results

In [26]:
run_configs = [
    {
        "name": "User2Vec (d=16)",
        "model_name": "doc2vec",
        "d_model": 16,
    },
    {
        "name": "User2Vec (d=32)",
        "model_name": "doc2vec",
        "d_model": 32,
    },
    {
        "name": "User2Vec (d=64)",
        "model_name": "doc2vec",
        "d_model": 64,
    },
    {
        "name": "AU2V (wo weight-tieing)",
        "model_name": "old-attentive",
        "d_model": 64,
    },
    {
        "name": "AU2V (d=16)",
        "model_name": "attentive",
        "d_model": 16,
    },
    {
        "name": "AU2V (d=32)",
        "model_name": "attentive",
        "d_model": 32,
    },
    {
        "name": "AU2V (d=64)",
        "model_name": "attentive",
        "d_model": 64,
    },
]

In [27]:
results = {}

for run_config in run_configs:
    results[run_config["name"]] = {"accuracy": []}
    model_config = ModelConfig(
        weight_decay=1e-8,
        max_embedding_norm=1,
        d_model=run_config["d_model"],
        lr=5e-5,
    )
    trainer_config = TrainerConfig(
        dataset_name="movielens",
        model_name=run_config["model_name"],
        load_dataset=False,
        save_dataset=False,
        load_model=False,
        ignore_saved_model=True,
        epochs=8,
    )

    print(model_config)
    print(trainer_config)

    dataset_manager = load_dataset_manager(
        dataset_name=trainer_config.dataset_name,
        dataset_dir=trainer_config.dataset_dir,
        data_dir="../data/",
        load_dataset=trainer_config.load_dataset,
        save_dataset=trainer_config.save_dataset,
        window_size=model_config.window_size,
    )
    model = load_model(
        dataset_manager=dataset_manager,
        trainer_config=trainer_config,
        model_config=model_config,
    )
    trainer = PyTorchTrainer(
        model=model,
        dataset_manager=dataset_manager,
        trainer_config=trainer_config,
        model_config=model_config,
    )

    def on_epoch_end(epoch):
        results[run_config["name"]]["accuracy"].append(
            calc_accuracy(
                model=model,
                num_item=dataset_manager.num_item,
                test_dataset=dataset_manager.test_datasets["test"],
                top_k=[10, 20, 30, 40, 50],
            )
        )
        torch.save(model, f"cache/model/movielens/{run_config['name']}-{epoch}.pt")

    losses = trainer.fit(on_epoch_end=on_epoch_end)
    results[run_config["name"]]["loss"] = losses
    print(results)

ModelConfig(d_model=16, init_embedding_std=0.2, max_embedding_norm=1, window_size=5, negative_sample_size=5, lr=5e-05, weight_decay=1e-08)
TrainerConfig(model_name='doc2vec', dataset_name='movielens', epochs=8, batch_size=64, verbose=False, ignore_saved_model=True, load_model=False, save_model=True, load_dataset=False, save_dataset=False, model_dir='cache/model/', dataset_dir='cache/dataset/', device='cpu')
dataset_manager does not exist at: cache/dataset/movielens.pickle, create dataset
num_seq: 6040, num_item: 3706, num_item_meta: 28, num_seq_meta: 30, num_item_meta_types: 3, num_seq_meta_types: 3
to_sequential_data start
to_sequential_data end


100%|██████████| 9190/9190 [00:37<00:00, 243.31it/s]


train 0.006517342345787048


100%|██████████| 2298/2298 [00:02<00:00, 886.70it/s]


valid 0.006486121344657288


...

valid 0.0051165665410231155


100%|██████████| 1591/1591 [00:17<00:00, 89.71it/s]


saved model to cache/model/movielens/attentive.pt


100%|██████████| 1591/1591 [00:17<00:00, 90.21it/s]

{'Accuracy@10': 0.11596480201131365, 'Accuracy@30': 0.11917033312382148, 'Accuracy@50': 0.12240100565681962}
{'User2Vec (d=16)': {'accuracy': [{'Accuracy@10': 0.03343808925204274, 'Accuracy@20': 0.03287240729101194, 'Accuracy@30': 0.03343808925204274, 'Accuracy@40': 0.03357950974230044, 'Accuracy@50': 0.03353865493400377}, {'Accuracy@10': 0.0399120050282841, 'Accuracy@20': 0.03966059082338152, 'Accuracy@30': 0.03951393253718835, 'Accuracy@40': 0.039754871150219986, 'Accuracy@50': 0.03922061596480201}, {'Accuracy@10': 0.05600251414204902, 'Accuracy@20': 0.0549654305468259, 'Accuracy@30': 0.05451498009637545, 'Accuracy@40': 0.053331238214959144, 'Accuracy@50': 0.0527215587680704}, {'Accuracy@10': 0.09220615964802012, 'Accuracy@20': 0.08846637335009429, 'Accuracy@30': 0.08403519798868636, 'Accuracy@40': 0.08062539283469516, 'Accuracy@50': 0.07865493400377122}, {'Accuracy@10': 0.1284098051539912, 'Accuracy@20': 0.12413576367064738, 'Accuracy@30': 0.11881416300020951, 'Accuracy@40': 0.11445

In [34]:
for method, result in results.items():
    a = max(map(lambda r: r["Accuracy@50"], result["accuracy"]))
    print(f"{method}, {a:.5}")

User2Vec (d=16), 0.14781
User2Vec (d=32), 0.16962
User2Vec (d=64), 0.17869
AU2V (wo weight-tieing), 0.18397
AU2V (wo normalize-norm), 0.034054
AU2V (d=16), 0.19119
AU2V (d=32), 0.19658
AU2V (d=64), 0.19883


In [28]:
models = ["attentive", "doc2vec", "old-attentive"]

for model_name in models:
    model = torch.load(f"cache/model/movielens/{model_name}.pt")

    print(
        model_name,
        calc_accuracy(
            model=model,
            num_item=dataset_manager.num_item,
            test_dataset=dataset_manager.test_datasets["test"],
            top_k=[10, 30, 50],
        ),
    )

100%|██████████| 1591/1591 [00:17<00:00, 88.78it/s]


attentive {'Accuracy@10': 0.11596480201131365, 'Accuracy@30': 0.11917033312382148, 'Accuracy@50': 0.12240100565681962}


100%|██████████| 1591/1591 [00:08<00:00, 197.95it/s]


doc2vec {'Accuracy@10': 0.18409805153991202, 'Accuracy@30': 0.18411900272365386, 'Accuracy@50': 0.1764676304211188}


100%|██████████| 1591/1591 [00:17<00:00, 89.57it/s]

old-attentive {'Accuracy@10': 0.08648648648648649, 'Accuracy@30': 0.0908652838885397, 'Accuracy@50': 0.08974230043997485}
